In [12]:
!pip install neurobench

import copy
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, ConcatDataset

from neurobench.benchmarks import Benchmark
from neurobench.datasets import MSWC
from neurobench.datasets.MSWC_IncrementalLoader import IncrementalFewShot
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


^C
Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site.py", line 617, in <module>
    main()
  File "/opt/conda/lib/python3.10/site.py", line 610, in main
    execsitecustomize()
  File "/opt/conda/lib/python3.10/site.py", line 549, in execsitecustomize
    import sitecustomize
  File "/root/.local/lib/python3.10/site-packages/sitecustomize.py", line 3, in <module>
    from log import Log
  File "/root/.local/lib/python3.10/site-packages/log.py", line 5, in <module>
    import google.auth
  File "/opt/conda/lib/python3.10/site-packages/google/auth/__init__.py", line 22, in <module>
    from google.auth._default import (
  File "/opt/conda/lib/python3.10/site-packages/google/auth/_default.py", line 28, in <module>
    import google.auth.transport._http_client
  File "/opt/conda/lib/python3.10/site-packages/google/auth/transport/__init__.py", line 28, in <module>

In [13]:
%load_ext tensorboard
# train and collect logs then call tensorboard
%tensorboard --logdir runs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
# To completely clean your tensorboard uncomment and run the following command.
#!rm -r runs

In [15]:
# data in repo root dir
ROOT = "./data/"

NUM_WORKERS = 4
BATCH_SIZE = 256

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device("cuda"):
    PIN_MEMORY = True
else:
    PIN_MEMORY = False
device

device(type='cpu')

In [17]:
SPIKING = False

In [18]:
from neurobench.preprocessing import MFCCPreProcessor, S2SPreProcessor

n_fft = 512
win_length = None
hop_length = 240
n_mels = 20
n_mfcc = 20

if SPIKING:
    encode = S2SPreProcessor(device, transpose=True)
    config_change = {"sample_rate": 48000,
                     "hop_length": 240}
    encode.configure(threshold=1.0, **config_change)
else:
    encode = MFCCPreProcessor(
        sample_rate=48000,
        n_mfcc=n_mfcc,
        melkwargs={
            "n_fft": n_fft,
            "n_mels": n_mels,
            "hop_length": hop_length,
            "mel_scale": "htk",
            "f_min": 20,
            "f_max": 4000,
        },
        device = device
    )

In [19]:
base_train_set = MSWC(root=ROOT, subset="base", procedure="training")

train_loader = DataLoader(base_train_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=True)

base_validation_set = MSWC(root=ROOT, subset="base", procedure="validation")

validation_loader = DataLoader(base_validation_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=True)

In [28]:
!pip install git+https://github.com/V0XNIHILI/TCN-library.git
!pip install torchsummary

from tcn_lib import TCN
from torchsummary import summary

model = TCN(20, 200, [(32,64)] * 3 + [(64,512)], [9] * 4, batch_norm=True, weight_norm=True, bottleneck=True, groups=8).to(device)

summary(model, (20, 200))

  Cloning https://github.com/V0XNIHILI/TCN-library.git to /tmp/pip-req-build-rcbkfat5
  Running command git clone --filter=blob:none --quiet https://github.com/V0XNIHILI/TCN-library.git /tmp/pip-req-build-rcbkfat5
  Resolved https://github.com/V0XNIHILI/TCN-library.git to commit ba449992fef2f43f310ba40571e32ff080f5a064
  Preparing metadata (setup.py) ... done
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 32, 200]             640
       BatchNorm1d-2              [-1, 32, 200]              64
              ReLU-3              [-1, 32, 200]               0
           Dropout-4              [-1, 32, 200]               0
       _WeightNorm-5                 [-1, 4, 9]               0
ParametrizedConv1d-6              [-1, 32, 208]           1,152
       _WeightNorm-7                 [-1, 4, 9]               0
       _WeightNorm-8                 [-1, 4, 9]              

In [21]:
!pip install -U numpy

^C
ERROR: Operation cancelled by user


In [22]:
import torch.optim as optim

# Create a writer to write to Tensorboard
writer = SummaryWriter()

# Some hyperparams
epochs = 25
#lr_decay = 1.05

# Create loss function and optimizer and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
#lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: lr_decay ** epoch)

for epoch in tqdm(range(epochs)):

    train_avg_loss = 0
    train_correct = 0
    train_total = 0

    validation_avg_loss = 0
    validation_correct = 0
    validation_total = 0

    for data, target in train_loader:
        data, target = encode((data.to(device), target.to(device)))
        data = data.squeeze()

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_avg_loss += loss.item()
        _, predicted = output.max(1)
        train_total += target.size(0)
        train_correct += predicted.eq(target).sum().item()
        
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = encode((data.to(device), target.to(device)))
            data = data.squeeze()

            output = model(data)
            loss = criterion(output, target)

            validation_avg_loss += loss.item()
            _, predicted = output.max(1)
            validation_total += target.size(0)
            validation_correct += predicted.eq(target).sum().item()

    train_loss, train_acc = train_avg_loss/len(train_loader), 100 * train_correct / train_total
    validation_loss, validation_acc = validation_avg_loss/len(validation_loader), 100 * validation_correct / validation_total

    print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.2f}")
    print(f"Epoch {epoch} - Validation Loss: {validation_loss:.4f} - Validation Acc: {validation_acc:.2f}")

    writer.add_scalars("Loss", {'Train': train_loss, 'Validation':validation_loss}, epoch)
    writer.add_scalars('Accuracy', {'Train': train_acc,'Validation':validation_acc} , epoch)

print('Finished Training')
writer.flush()
writer.close()

  0%|          | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
  0%|          | 0/50 [00:11<?, ?it/s]


KeyboardInterrupt: 